# Shohei Ohtani Batting Stats

- 大谷翔平の打撃成績を可視化する例
- サンプルデータはkm/h & m 単位になっていますが, [オリジナルデータ](https://baseballsavant.mlb.com/statcast_search)はmile/h & feetなのでご注意ください

In [64]:
import pandas as pd

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [65]:
# データ読み込み. game_dateは予めデータ型を変更

df = pd.read_csv('dataset/20221005_baseball-savant-shohei-ohtani-atbat.csv', header=0)

df['game_date'] = pd.to_datetime(df['game_date'])
df.head(10)

,game_date,game_month,game_pk,inning,pitcher_name,p_throws,batter_name,stand,at_bat_number,pitch_number,pitch_name,effective_speed,release_spin_rate,release_extension,type,events,description,des,zone,bb_type,launch_speed_angle,hit_location,launch_speed,launch_angle,hit_distance_sc,pfx_x,pfx_z,plate_x,plate_z,release_pos_x,release_pos_z,effective_speed_kh,release_extension_cm,pfx_x_cm,pfx_z_cm,plate_x_cm,plate_z_cm,release_pos_x_cm,release_pos_z_cm,launch_speed_angle_category,launch_speed_kh,hit_distance_sc_m,hc_x,hc_y,hc_x_0,hc_y_0,hc_x_ft,hc_y_ft,hc_x_0_ft,hc_y_0_ft,hc_x_m,hc_y_m,hc_x_0_m,hc_y_0_m,events_summary
0,2022-10-05,10,662421,1.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,3,1,4-Seam Fastball,94.5,2276.0,7.1,S,NaN,called_strike,Shohei Ohtani singles on a line drive to right...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.78,1.29,-0.64,2.98,2.58,5.53,151.20,216.41,23.77,39.32,-19.51,90.83,78.64,168.55,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
1,2022-10-05,10,662421,1.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,3,2,Slider,80.5,2153.0,7.0,X,single,hit_into_play,Shohei Ohtani singles on a line drive to right...,13.0,line_drive,4.0,NaN,73.2,22.0,229.0,-1.64,-0.17,-1.09,2.19,2.90,5.22,128.80,213.36,-49.99,-5.18,-33.22,66.75,88.39,159.11,Flare/Burner,117.12,69.80,163.02,102.86,37.60,95.41,407.55,257.15,94.00,238.53,124.22,78.38,28.65,72.70,hits
2,2022-10-05,10,662421,3.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,18,1,Slider,81.1,2264.0,7.3,S,NaN,foul,Shohei Ohtani grounds out to first baseman Der...,2.0,NaN,NaN,NaN,80.4,69.0,179.0,-1.74,-0.46,0.08,3.11,3.09,5.24,129.76,222.50,-53.04,-14.02,2.44,94.79,94.18,159.72,Unknown,128.64,54.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
3,2022-10-05,10,662421,3.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,18,2,4-Seam Fastball,93.4,2157.0,7.0,B,NaN,ball,Shohei Ohtani grounds out to first baseman Der...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,0.82,1.32,0.82,4.15,2.77,5.56,149.44,213.36,24.99,40.23,24.99,126.49,84.43,169.47,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
4,2022-10-05,10,662421,3.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,18,3,4-Seam Fastball,95.4,2317.0,7.2,B,NaN,ball,Shohei Ohtani grounds out to first baseman Der...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.83,1.29,-0.83,3.50,2.76,5.52,152.64,219.46,25.30,39.32,-25.30,106.68,84.12,168.25,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
5,2022-10-05,10,662421,3.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,18,4,4-Seam Fastball,93.7,2264.0,7.1,S,NaN,called_strike,Shohei Ohtani grounds out to first baseman Der...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.81,1.31,-0.90,2.67,2.62,5.45,149.92,216.41,24.69,39.93,-27.43,81.38,79.86,166.12,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
6,2022-10-05,10,662421,3.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,18,5,4-Seam Fastball,96.5,2336.0,7.2,S,NaN,foul,Shohei Ohtani grounds out to first baseman Der...,4.0,NaN,NaN,NaN,78.2,16.0,176.0,0.71,1.26,-0.41,2.58,2.59,5.59,154.40,219.46,21.64,38.40,-12.50,78.64,78.94,170.38,Unknown,125.12,53.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
7,2022-10-05,10,662421,3.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,18,6,Slider,81.1,2317.0,7.2,X,field_out,hit_into_play,Shohei Ohtani grounds out to first baseman Der...,5.0,ground_ball,2.0,NaN,82.2,-1.0,33.0,-1.77,-0.42,0.16,2.34,2.89,5.18,129.76,219.46,-53.95,-12.80,4.88,71.32,88.09,157.89,Topped,131.52,10.06,154.75,165.38,29.33,32.89,386.88,413.45,73.32,82.23,117.92,126.02,22.35,25.06,other
8,2022-10-05,10,662421,6.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,38,1,Curveball,76.6,2008.0,7.1,B,NaN,ball,Shohei Ohtani strikes out swinging.,11.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.39,-0.60,-1.01,3.35,2.39,5.72,122.56,216.41,-42.37,-18.29,-30.78,102.11,72.85,174.35,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other
9,2022-10-05,10,662421,6.0,"Waldichuk, Ken",L,"Ohtani, Shohei",L,38,2,Slider,81.0,2244.0,7.1,S,NaN,foul,Shohei Ohtani strikes out swinging.,9.0,NaN,NaN,NaN,75.7

In [66]:
# 念の為データ型を確認
df.dtypes

game_date                      datetime64[ns]
game_month                              int64
game_pk                                 int64
inning                                float64
pitcher_name                           object
p_throws                               object
batter_name                            object
stand                                  object
at_bat_number                           int64
pitch_number                            int64
pitch_name                             object
effective_speed                       float64
release_spin_rate                     float64
release_extension                     float64
type                                   object
events                                 object
description                            object
des                                    object
zone                                  float64
bb_type                                object
launch_speed_angle                    float64
hit_location                      

## Color Map定義

- 球種, イベントに合わせてカラーマップを定義
- 色は基本的に[オリジナルデータ](https://baseballsavant.mlb.com/statcast_search)の掲載元に合わせて定義
- Plotlyで描画するときに引数として渡す


In [67]:
from typing import Dict
class Color:
    EVENTS: Dict[str, str] = {
        'catcher_interf': 'rgb(255, 255, 255)',
        'caught_stealing_2b': 'rgb(255, 255, 255)',
        'caught_stealing_3b': 'rgb(255, 255, 255)',
        'caught_stealing_home': 'rgb(255, 255, 255)',
        'double': 'rgb(119, 94, 240)',
        'double_play': 'rgb(194, 194, 194)',
        'field_error': 'rgb(255, 255, 255)',
        'field_out': 'rgb(194, 194, 194)',
        'fielders_choice': 'rgb(255, 255, 255)',
        'fielders_choice_out': 'rgb(255, 255, 255)',
        'force_out': 'rgb(194, 194, 194)',
        'game_advisory': 'rgb(255, 255, 255)',
        'grounded_into_double_play': 'rgb(194, 194, 194)',
        'hit_by_pitch': 'rgb(255, 255, 255)',
        'home_run': 'rgb(214, 43, 126)',
        'other_out': 'rgb(194, 194, 194)',
        'pickoff_1b': 'rgb(255, 255, 255)',
        'pickoff_2b': 'rgb(255, 255, 255)',
        'pickoff_3b': 'rgb(255, 255, 255)',
        'pickoff_caught_stealing_3b': 'rgb(255, 255, 255)',
        'pickoff_caught_stealing_home': 'rgb(255, 255, 255)',
        'sac_bunt': 'rgb(255, 255, 255)',
        'sac_fly': 'rgb(255, 255, 255)',
        'sac_fly_double_play': 'rgb(194, 194, 194)',
        'single': 'rgb(253, 97, 0)',
        'strikeout': 'rgb(194, 194, 194)',
        'strikeout_double_play': 'rgb(194, 194, 194)',
        'triple': 'rgb(255, 177, 2)',
        'triple_play': 'rgb(194, 194, 194)',
        'walk': 'rgb(255, 255, 255)',
        'wild_pitch': 'rgb(255, 255, 255)',
    }

    BB_TYPE: Dict[str, str] = {
        'fly_ball': 'rgb(80, 95, 58)',
        'ground_ball': 'rgb(183, 21, 131)',
        'line_drive': 'rgb(236, 236, 76)',
        'popup': 'rgb(222, 30, 36)',
    }

    LAUNCH_SPEED_ANGLE: Dict[str, str] = {
        'Weak': 'rgb(255, 255, 255)',
        'Topped': 'rgb(194, 194, 194)',
        'Under': 'rgb(253, 97, 0)',
        'Flare/Burner': 'rgb(119, 94, 240)',
        'Solid Contact': 'rgb(255, 177, 2)',
        'Barrel': 'rgb(214, 43, 126)',
    }

    PITCH_TYPE: Dict[str, str] = {
        '2-Seam Fastball': 'rgb(254, 157, 0)',
        '4-Seam Fastball': 'rgb(210, 46, 73)',
        'Changeup': 'rgb(28, 191, 58)',
        'Curveball': 'rgb(0, 210, 236)',
        'Cutter': 'rgb(148, 63, 44)',
        'Eephus': 'rgb(255, 255, 255)',
        'Fastball': 'rgb(210, 46, 73)',
        'Knuckle Curve': 'rgb(98, 54, 204)',
        'Knuckleball': 'rgb(0, 0, 0)',
        'Sinker': 'rgb(254, 157, 0)',
        'Slider': 'rgb(237, 231, 22)',
        'Split-Finger': 'rgb(58, 172, 172)',
    }

    DESCRIPTION: Dict[str, str] = {
        'ball': 'rgb(32, 119, 180)',
        'blocked_ball': 'rgb(32, 119, 180)',
        'bunt_foul_tip': 'rgb(42, 160, 45)',
        'called_strike': 'rgb(254, 127, 11)',
        'foul': 'rgb(42, 160, 45)',
        'foul_bunt': 'rgb(42, 160, 45)',
        'foul_tip': 'rgb(42, 160, 45)',
        'hit_by_pitch': 'rgb(0, 0, 0)',
        'hit_into_play': 'rgb(149, 102, 188)',
        'missed_bunt': 'rgb(255, 255, 255)',
        'pitchout': 'rgb(255, 255, 255)',
        'swinging_strike': 'rgb(227, 120, 193)',
        'swinging_strike_blocked': 'rgb(227, 120, 193)',
    }

## 投球データの可視化

- どこに投げられているか
- 球種毎, 結果毎にプロット

In [68]:
# グラフ描画用の関数

import plotly.express as px
import plotly.graph_objects as go

# ゾーンの横幅と縦幅
PLATE_ZONE_X = (-100, 100)
PLATE_ZONE_Y = (0, 225)

# ストライクゾーン定義（本当は上下可変ですがわかりやすいように固定にしています）
PLATE_STRIKE_ZONE_X = [-28.9, -28.9, 28.9, 28.9, -28.9]
PLATE_STRIKE_ZONE_Y = [48, 104, 104, 48, 48]

HOVER = [
    "game_date",
    "batter_name",
    "pitcher_name",
    "effective_speed_kh",
    "pitch_name",
    "hit_distance_sc_m",
    "launch_speed_kh",
    "launch_angle",
    "des",
    "description"
]
LEGEND = dict(
    yanchor="top",
    y=1,
    xanchor="left",
    x=0,
    orientation="h"
)
def strike_zone(name: str = 'strike zone') -> go.Scatter:
    return go.Scatter(
        name=name,
        x=PLATE_STRIKE_ZONE_X,
        y=PLATE_STRIKE_ZONE_Y,
        line_color='red'
    )


def plate(df: pd.DataFrame, title: str = 'Pitch on plate(Umpire view)',
            hover: str = 'pitch_name', colormap: Dict[str, str] = Color.PITCH_TYPE) -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="plate_x_cm",
        y="plate_z_cm",
        color=hover,
        color_discrete_map=colormap,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(strike_zone())
    fig.update_layout(
        title=title,
        autosize=False,
        width=800,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=PLATE_ZONE_X, title='plate_x(cm)', dtick=25)
    fig.update_yaxes(range=PLATE_ZONE_Y, title='plate_x(cm)', dtick=25)
    return fig


In [69]:
# 早速可視化してみる

_fig = plate(df)
_fig.show()

In [70]:
# 結果ごとに可視化

_fig = plate(df, hover='events', colormap=Color.EVENTS)
_fig.show()

In [71]:
# 一試合に絞る

_fig = plate(df[df['game_date'] == '2022-09-17'])
_fig.show()

## 打球の到達位置

内野のダイヤモンドおよび外野のどこに飛んだかの記録

In [72]:
DIAMOND_X_M = [0, -19.2, 0, 19.2, 0]
DIAMOND_Y_M = [0, 19.2, 38.4, 19.2, 0]
FIELD_X_M = [-100, 100]
FIELD_Y_M = [0, 200]
FIELD_LINE = {
    'x': [-100, 0, 100],
    'y': [100, 0, 100]
}

def hit_location(df: pd.DataFrame, title: str = 'Hit Location',
                    hover: str = 'events', colormap: Dict[str, str] = Color.EVENTS) -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="hc_x_0_m",
        y="hc_y_0_m",
        color=hover,
        color_discrete_map=colormap,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.add_trace(go.Scatter(
        name='diamond',
        x=DIAMOND_X_M,
        y=DIAMOND_Y_M,
        line_color='black'
    )
    )
    fig.add_trace(go.Scatter(
        name='line',
        x=FIELD_LINE['x'],
        y=FIELD_LINE['y'],
        line_color='black',
    )
    )

    fig.update_layout(
        title=title,
        legend=LEGEND,
        autosize=False,
        width=800,
        height=800,
        legend_title=None
    )
    fig.update_xaxes(range=FIELD_X_M, title='hc_x(m)', dtick=40)
    fig.update_yaxes(range=FIELD_Y_M, title='hc_y(m)', dtick=40)
    return fig

In [73]:
# 打球が飛んだイベントのみ抽出して可視化

_fig = hit_location(df[df['type'] == 'X'])
_fig.show()

In [74]:
# ホームランのみに着目してみる

_fig = hit_location(df[df['events'] == 'home_run'])
_fig.show()

## 打球速度×角度による分類

- 打球速度が速いかつ, いい感じの角度で飛ぶと長打の確率が高くなる
- これを「バレルゾーン」と呼んでいる
- 横軸を打球速度, 縦軸を角度にして可視化してみる

In [75]:
BARREL_SPEED = [0, 200]
BARREL_ANGLE = [-80, 160]

def barrel(df: pd.DataFrame, title: str = 'Launch speed & angle', hover: str = 'events', colormap: Dict[str, str] = Color.EVENTS) -> go.Figure:
    fig = px.scatter(
        df.sort_values(hover),
        x="launch_speed_kh",
        y="launch_angle",
        color=hover,
        color_discrete_map=colormap,
        hover_name=hover,
        hover_data=HOVER
    )
    fig.update_layout(
        title=title,
        autosize=False,
        width=1200,
        height=800,
        legend=LEGEND,
        legend_title=None
    )
    fig.update_xaxes(range=BARREL_SPEED, title='launch_speed_kh(km/h)', dtick=25)
    fig.update_yaxes(range=BARREL_ANGLE, title='launch_angle', dtick=20)
    return fig


In [76]:
# 速度と角度に着目した可視化

_fig = barrel(df)
_fig.show()

In [77]:
# ホームランだけ残したい

_fig = barrel(df[df['events'] == 'home_run'])
_fig.show()